### Import bibliothèques

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import os
 
def lister_fichiers(repertoire):
    return os.listdir(repertoire)
 
liste_avions=lister_fichiers('IMAGES/Imagettes/avions')
liste_drones=lister_fichiers('IMAGES/Imagettes/drones')
liste_helicopter=lister_fichiers('IMAGES/Imagettes/hélicoptères')
liste_missiles=lister_fichiers('IMAGES/Imagettes/missiles')
liste_oiseaux=lister_fichiers('IMAGES/Imagettes/oiseaux')



 ### Fonction de comparaison ORB :

In [3]:
def comparaisonORB (path1, path2):
    img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    
    img2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

    orb = cv2.ORB_create()
    points1, descripteursImg1 = orb.detectAndCompute(img1, None)
    points2, descripteursImg2 = orb.detectAndCompute(img2, None)
    
    algoBF = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    paires_corresp = algoBF.knnMatch(descripteursImg1, descripteursImg2, k=5)

    Matches_tri = sorted(paires_corresp, key=lambda x:x[0].distance)

    Matches_tri = Matches_tri[:10]
    
    sumDistances = 0

    for group in Matches_tri:
        for match in group :
            sumDistances += match.distance

    return sumDistances

    
    

In [4]:
DicoComparaison = {"Avions" : [], "Drones" : [], "Hélicoptères" : [], "Missiles" : [], "Oiseaux" : []}

pathUser = str(input("Rentrez le path de votre image : "))
for i in range(10):
    pathAvions = 'IMAGES/Imagettes/avions/' + liste_avions[i]
    nb=comparaisonORB(pathUser, pathAvions)
    DicoComparaison["Avions"].append(nb)
    pathDrones = 'IMAGES/Imagettes/drones/' + liste_drones[i]
    nb=comparaisonORB(pathUser, pathDrones)
    DicoComparaison["Drones"].append(nb)
    pathHelico = 'IMAGES/Imagettes/hélicoptères/' + liste_helicopter[i]
    nb=comparaisonORB(pathUser, pathHelico)
    DicoComparaison["Hélicoptères"].append(nb)
    pathMissiles = 'IMAGES/Imagettes/missiles/' + liste_missiles[i]
    nb=comparaisonORB(pathUser, pathMissiles)
    DicoComparaison["Missiles"].append(nb)
    pathOiseaux = 'IMAGES/Imagettes/oiseaux/' + liste_oiseaux[i]
    nb=comparaisonORB(pathUser, pathOiseaux)
    DicoComparaison["Oiseaux"].append(nb)

print("Avions : ", np.sum(DicoComparaison["Avions"]),"\nOiseaux : ", np.sum(DicoComparaison["Oiseaux"]), "\nDrones : ", np.sum(DicoComparaison["Drones"]), "\nHelicoptère : ", np.sum(DicoComparaison["Hélicoptères"]))


Avions :  24504.0 
Oiseaux :  24963.0 
Drones :  24615.0 
Helicoptère :  26876.0
